In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [33]:
from bs4 import BeautifulSoup

# Read the HTML file
with open("Amazon.in _ laptops.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# List to store extracted data
products = []

# Find all relevant product blocks
product_divs = soup.find_all("div", class_="a-section a-spacing-small a-spacing-top-small")

for div in product_divs:
    try:
        inner_div = div.find("div", class_="a-section a-spacing-none puis-padding-right-small s-title-instructions-style")
        if not inner_div:
            continue  # Skip if inner structure not found

        # Extract product name
        try:
            name = inner_div.find("h2").get("aria-label", "")
        except:
            name = ""

        # Extract price
        try:
            price = div.find("span", class_="a-price-whole").text.strip()
        except:
            price = ""

        # Extract review stars
        try:
            reviews = div.find("span", class_="a-icon-alt").text.strip()
        except:
            reviews = ""

        # Extract rating count
        try:
            rating_tag = div.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style")
            rating = rating_tag.find("span", class_="a-size-base s-underline-text").text.strip()
        except:
            rating = ""

        products.append({
            "name": name,
            "price": price,
            "reviews": reviews,
            "rating": rating
        })

    except Exception as e:
        print(f"Error parsing product: {e}")

# Print the extracted data
for product in products:
    print(product)


{'name': 'Sponsored Ad - Acer Aspire Lite 13th Gen Intel Core i3-1305U Thin and Light Premium Laptop (Windows 11 Home/8 GB RAM/512GB SSD/36 WHR) AL15-53 with 39.62cm (15.6") Full HD Display, Metal Body, Steel Gray, 1.59 KG', 'price': '31,990', 'reviews': '3.8 out of 5 stars', 'rating': ''}
{'name': 'Sponsored Ad - Dell Inspiron 3530 Thin & Light Laptop, 13th Gen Intel Core i5-1334U Processor, 16GB, 1TB SSD, 15.6" (39.62cm) FHD Display, Backlit Keyboard, Windows 11 + MSO\'24 & 15 Month McAfee, Silver, 1.62kg', 'price': '57,490', 'reviews': '3.7 out of 5 stars', 'rating': ''}
{'name': 'HP 255 G10 AMD Ryzen 5 Quad Core 7520U 2.8Ghz Upto 4.3Ghz - (8 GB/512 GB SSD) 255 G10 Thin and Light Laptop (15.6 inch, Matt Silver, 1.45 kg)', 'price': '29,828', 'reviews': '3.2 out of 5 stars', 'rating': '18'}
{'name': 'Apple MacBook Air Laptop with M2 chip: 34.46 cm (13.6-inch) Liquid Retina Display, 8GB RAM, 512GB SSD Storage, Backlit Keyboard, 1080p FaceTime HD Camera. Works with iPhone/iPad; Space Gr

In [35]:
df = pd.DataFrame(products)
df.to_csv("amazon_laptops.csv", index=False, encoding="utf-8")
print("Data saved to amazon_laptops.csv")

Data saved to amazon_laptops.csv


In [3]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_amazon(product_name, max_results=25000):
    driver = webdriver.Chrome()
    url = f'https://www.amazon.in/s?k={product_name.replace(" ", "+")}'
    driver.get(url)

    with open(f'{product_name}_amazon_data.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Product Name', 'Price (₹)', 'Rating', 'Number of Ratings'])
        
        total_results = 0

        while total_results < max_results:
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h2.a-size-medium span'))
                )

                products = driver.find_elements(By.CSS_SELECTOR, 'h2.a-size-medium span')
                prices = driver.find_elements(By.CSS_SELECTOR, 'span.a-price-whole')
                ratings = driver.find_elements(By.CSS_SELECTOR, 'span.a-icon-alt')
                num_ratings = driver.find_elements(By.CSS_SELECTOR, 'span.a-size-base.s-underline-text')

                for i in range(len(products)):
                    product_text = products[i].text.strip() if i < len(products) else 'Not Found'
                    price_text = prices[i].text.strip() if i < len(prices) else 'Not Found'
                    rating_text = ratings[i].get_attribute('innerHTML').strip() if i < len(ratings) else 'Not Found'
                    num_ratings_text = num_ratings[i].text.strip() if i < len(num_ratings) else 'Not Found'

                    writer.writerow([product_text, price_text, rating_text, num_ratings_text])
                    total_results += 1

                    if total_results >= max_results:
                        print("Reached maximum result limit.")
                        break

                print(f"Scraped {len(products)} products on current page! Total results: {total_results}")

                # Check for the next page
                next_button = driver.find_elements(By.CSS_SELECTOR, 'a.s-pagination-next')
                if not next_button or 's-pagination-disabled' in next_button[0].get_attribute('class'):
                    print("Reached last page. Exiting...")
                    break

                next_button[0].click()
                time.sleep(2)

            except Exception as e:
                print(f"Error: {e}")
                break

    driver.quit()
    print(f"Scraping for {product_name} completed with {total_results} results!")

# Start scraping for laptops
scrape_amazon('laptop')

Error: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.178)
Stacktrace:
	GetHandleVerifier [0x00007FF6788B4C25+3179557]
	(No symbol) [0x00007FF6785188A0]
	(No symbol) [0x00007FF6783A91CA]
	(No symbol) [0x00007FF678394F85]
	(No symbol) [0x00007FF6783B9F94]
	(No symbol) [0x00007FF67842F9DF]
	(No symbol) [0x00007FF67844FBE2]
	(No symbol) [0x00007FF678427A03]
	(No symbol) [0x00007FF6783F06D0]
	(No symbol) [0x00007FF6783F1983]
	GetHandleVerifier [0x00007FF6789167CD+3579853]
	GetHandleVerifier [0x00007FF67892D1D2+3672530]
	GetHandleVerifier [0x00007FF678922153+3627347]
	GetHandleVerifier [0x00007FF67868092A+868650]
	(No symbol) [0x00007FF678522FFF]
	(No symbol) [0x00007FF67851F4A4]
	(No symbol) [0x00007FF67851F646]
	(No symbol) [0x00007FF67850EAA9]
	BaseThreadInitThunk [0x00007FF868BDE8D7+23]
	RtlUserThreadStart [0x00007FF86A2714FC+44]

Scraping for laptop completed with 0 re